<a href="https://colab.research.google.com/github/ifrahsaleem/viterbi/blob/master/viterbi_implementation_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np 

states = ('Exon', 'Splice', 'Intron' )
seq="CTTCATGTGAAAGCAGACGTAAGTCA"
obs = tuple(seq)
start_probability = {'Exon': 1.0, 'Splice': 0.0, 'Intron': 0.0}
transition_probability = {
   'Exon' : {'Exon': 0.9, 'Splice': 0.1, 'Intron': 0.0, 'end': 0.0},
   'Splice' : {'Exon': 0.0, 'Splice': 0.0, 'Intron': 1.0, 'end': 0.0},
   'Intron' : {'Exon': 0.0, 'Splice': 0.0, 'Intron': 0.9, 'end': 0.1}
   }
emission_probability = {'Exon' : {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
                        'Splice' : {'A': 0.05, 'C': 0.0, 'G': 0.95, 'T': 0.0},
                        'Intron' : {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4},}

def print_path_probability_matrix(P,states):
    for q in states: print(q+ ":\t" + "\t".join(("%.2e" % (p)) for p in P[q]))
      
def viterbi(obs, states, phi , trans_p, emit_p): #phi is start prob
  P = {}
  backpointer = {}
  T=len(obs) 
  
  #initializationstep
  start_probability = phi
  for q in states:
    P[q] = [phi[q] * emit_p[q][obs[0]]]
    backpointer[q] = ["Start"]
  
  #recursion step 
  #run viterbi when T > 0 
  for t in range(1,T):
    for current_q in states:
      (delta_i, state)=max(((P[q][t-1] * transition_probability[q][current_q] * emission_probability[current_q][obs[t]]), q) for q in states)
      P[current_q].append(delta_i)
      backpointer[current_q].append(state)
  
  #finalizationstep
  P.update({'End' : [] })
  backpointer.update({'End' : [] })
  (delta_i, state)=max(((P[q][-1] * transition_probability[q]['end']), q) for q in states)
  P['End'].append(delta_i)
  backpointer['End'].append(state)
  
 
  #bestpath
  Exon = backpointer['Exon']
  Splice = backpointer['Splice']
  Intron = backpointer['Intron']
  path = []
  #path.append('Start')
  
  count = 0
  y = 'a' 
  if backpointer['End'][0] == 'Intron':
      path.append('Intron')
      for r in range(T-1,0,-1):
        if Intron[r] == 'Intron':
          path.append('Intron')    
          count += 1
        elif Intron[r] != 'Intron':
          y = Intron[r]
          path.append(Intron[r])
          count += 1
          break 
       
  for r in range(T-1-count, 0, -1):
    if y == 'Splice':
      path.append('Exon')
      
  path.append('Start')
  path.reverse()
  path.append('End')
  
  print_path_probability_matrix(P,states)
  print("\n")
  
  for delta_i in P['End']:  
    print("probability of the best path to observe the sequence: %.2e " % (delta_i)) 
    print("\n")
    print("the best path:  %s"  % "\t" .join(path[::1]))

viterbi(obs, states, start_probability, transition_probability, emission_probability)


Exon:	2.50e-01	5.63e-02	1.27e-02	2.85e-03	6.41e-04	1.44e-04	3.24e-05	7.30e-06	1.64e-06	3.69e-07	8.31e-08	1.87e-08	4.21e-09	9.47e-10	2.13e-10	4.79e-11	1.08e-11	2.43e-12	5.46e-13	1.23e-13	2.76e-14	6.22e-15	1.40e-15	3.15e-16	7.08e-17	1.59e-17
Splice:	0.00e+00	0.00e+00	0.00e+00	0.00e+00	1.42e-05	0.00e+00	1.37e-05	0.00e+00	6.93e-07	8.21e-09	1.85e-09	4.16e-10	1.78e-09	0.00e+00	4.73e-12	2.02e-11	2.40e-13	0.00e+00	2.31e-13	0.00e+00	6.14e-16	1.38e-16	5.91e-16	0.00e+00	0.00e+00	3.54e-19
Intron:	0.00e+00	0.00e+00	0.00e+00	0.00e+00	0.00e+00	5.70e-06	5.13e-07	5.48e-06	4.93e-07	2.77e-07	9.98e-08	3.59e-08	3.23e-09	2.91e-10	1.05e-10	9.43e-12	8.10e-12	7.29e-13	6.56e-14	9.22e-14	3.32e-14	1.20e-14	1.08e-15	3.87e-16	3.49e-17	1.25e-17


probability of the best path to observe the sequence: 1.25e-18 


the best path:  Start	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Exon	Splice	Intron	Intron	Intron	Intron	Intron	Intron	Intron	End
